In [1]:
import sys

sys.path.insert(0, '/home/jupyter/AdversarialRobustness')
sys.path.insert(0, '/home/jupyter/AdversarialRobustness/vim')

In [2]:
#!pip install torch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1 --index-url https://download.pytorch.org/whl/cu118
#!pip install -r AdversarialRobustness/vim_requirements.txt
#!pip install causal-conv1d==1.1.0
#!pip install mamba-ssm==1.1.1
#!pip install timm
#!pip install -r AdversarialRobustness/requirements.txt

In [3]:
# Add sbin to path for Trigon - fix of common Debian error
import os
original_path = os.environ.get('PATH')
os.environ['PATH'] = original_path + ':/sbin'

In [4]:
#import mamba_ssm
#mamba_ssm.__file__

In [5]:
# Replace pip mamba_ssm with author modification
#!rm -rf /opt/conda/envs/pytorch/lib/python3.10/site-packages/mamba_ssm
#!cp -r ./mamba_ssm /opt/conda/envs/pytorch/lib/python3.10/site-packages/mamba_ssm

In [6]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
import requests
from matplotlib import pyplot as plt
from torchvision import datasets
from transformers import ViTImageProcessor, ViTForImageClassification

from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

import loadVim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# Load util functions to generate adversarial dataset
from captum.robust import FGSM, MinParamPerturbation
from genAdvDataset import fgsmAttack, calcImageDistanceMetrics, saveImageTensor, genImageNetteConvDict

In [8]:
# Use ViT for config, as Vim doesn't contain metadata
confproxymod = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")
netteToImageNetConv = genImageNetteConvDict(confproxymod)

# Generate Baseline Adversarial Dataset

In [9]:
# Load Model and prepare processor
model, processor = loadVim.loadVim()
normalize = transforms.Normalize(mean=processor.image_mean, std=processor.image_std)

# Predict function
def pred(inputs, model, preprocess=True):
    # preprocess the image
    if preprocess:
        inputs = normalize(inputs)

    # If input is not batched, batch it
    if len(inputs.shape) < 4:
        inputs = inputs.unsqueeze(0)

    outputs = model(inputs.to(device))
    logits = outputs.logits
    val, idx = torch.max(logits, dim=1)
    return idx

# Load dataset
def process_image(image):
    return processor(image, return_tensors="pt").pixel_values.squeeze(0)

data = datasets.ImageFolder('./AdversarialRobustness/imagenette/val', transform=transforms.Lambda(process_image))

dataLoader = torch.utils.data.DataLoader(data, batch_size=1, shuffle=False, num_workers=0)

# Adversarial attack
# Min Param Perturbation works only with batch size 1 for our purposes
min_pert = MinParamPerturbation(
                                forward_func = lambda *args, **kwargs: model(*args, **kwargs).logits,
                                attack = fgsmAttack,
                                arg_name = 'epsilon',
                                mode = "binary",
                                arg_min = 0.001,
                                arg_max = 1.5,
                                arg_step = 0.001,
                                preproc_fn = normalize,
                                apply_before_preproc=True
                            )

alt_im, min_eps, inputs = None, None, None

path_adversarial_dataset = './AdversarialRobustness/data_adv_vim'
os.makedirs(path_adversarial_dataset, exist_ok=True)

log = []

for i, (input, label) in enumerate(dataLoader):
    input = input.to(device)
    # Convert label item to ImageNet label
    label = torch.tensor([netteToImageNetConv[label.item()]]).to(device)
    label = label.to(device)

    prediction = pred(input, model)

    if prediction == label:
        # We only care for adversarial examples in the case when the model could correctly predict otherwise
        attack_kwargs={'model':model,'true_label':label}

        alt_im, min_eps = min_pert.evaluate(input, attack_kwargs=attack_kwargs, target=label)

        altpred = pred(alt_im, model)
        print(f"adv_{i} is a {model.config.id2label[label.item()]} but classifies as {model.config.id2label[altpred.item()]} with epsilon {min_eps}")

        mse, ssim_val, linf = calcImageDistanceMetrics(input, alt_im)
        log.append({"index": i, "label": label.item(), "prediction": altpred.item(), "epsilon": min_eps, "mse": mse, "ssim": ssim_val, "linf": linf})

        # Save the adversarial example to new dataset
        saveImageTensor(alt_im, path_adversarial_dataset, label, f"adv_{i}")
    else:
        print(f"Incorrect classification, label: {label}, prediction: {prediction}, skipping...")

# Pickle the log
with open(f"{path_adversarial_dataset}/log.pkl", "wb") as f:
    pickle.dump(log, f)

/opt/conda/envs/pytorch/lib/python3.10/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(


adv_0 is a tench, Tinca tinca but classifies as French horn, horn with epsilon 0.191
adv_1 is a tench, Tinca tinca but classifies as gasmask, respirator, gas helmet with epsilon 0.20400000000000001


KeyboardInterrupt: 

# Generate Celeb Adversarial Dataset

In [ ]:
# Load Model and prepare processor

model, processor = loadVim.prepareDownstreamModel()
normalize = transforms.Normalize(mean=processor.image_mean, std=processor.image_std)

# Predict function
def pred(inputs, model, preprocess=True):
    # preprocess the image
    if preprocess:
        inputs = normalize(inputs)

    # If input is not batched, batch it
    if len(inputs.shape) < 4:
        inputs = inputs.unsqueeze(0)

    outputs = model(inputs.to(device))
    logits = outputs.logits
    val, idx = torch.max(logits, dim=1)
    return idx

# Load dataset
def process_image(image):
    return processor(image, return_tensors="pt").pixel_values.squeeze(0)

data = datasets.ImageFolder('./AdversarialRobustness/imagenette/val', transform=transforms.Lambda(process_image))

dataLoader = torch.utils.data.DataLoader(data, batch_size=1, shuffle=False, num_workers=0)

# Adversarial attack
# Min Param Perturbation works only with batch size 1 for our purposes
min_pert = MinParamPerturbation(
                                forward_func = lambda *args, **kwargs: model(*args, **kwargs).logits,
                                attack = fgsmAttack,
                                arg_name = 'epsilon',
                                mode = "binary",
                                arg_min = 0.001,
                                arg_max = 1.5,
                                arg_step = 0.001,
                                preproc_fn = normalize,
                                apply_before_preproc=True
                            )

alt_im, min_eps, inputs = None, None, None

path_adversarial_dataset = './AdversarialRobustness/data_adv_vim'
os.makedirs(path_adversarial_dataset, exist_ok=True)

log = []

for i, (input, label) in enumerate(dataLoader):
    input = input.to(device)
    # Convert label item to ImageNet label
    label = torch.tensor([netteToImageNetConv[label.item()]]).to(device)
    label = label.to(device)

    prediction = pred(input, model)

    if prediction == label:
        # We only care for adversarial examples in the case when the model could correctly predict otherwise
        attack_kwargs={'model':model,'true_label':label}

        alt_im, min_eps = min_pert.evaluate(input, attack_kwargs=attack_kwargs, target=label)

        altpred = pred(alt_im, model)
        print(f"adv_{i} is a {model.config.id2label[label.item()]} but classifies as {model.config.id2label[altpred.item()]} with epsilon {min_eps}")

        mse, ssim_val, linf = calcImageDistanceMetrics(input, alt_im)
        log.append({"index": i, "label": label.item(), "prediction": altpred.item(), "epsilon": min_eps, "mse": mse, "ssim": ssim_val, "linf": linf})

        # Save the adversarial example to new dataset
        saveImageTensor(alt_im, path_adversarial_dataset, label, f"adv_{i}")
    else:
        print(f"Incorrect classification, label: {label}, prediction: {prediction}, skipping...")

# Pickle the log
with open(f"{path_adversarial_dataset}/log.pkl", "wb") as f:
    pickle.dump(log, f)